In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd  # Import pandas for saving the data

# Setup Chrome options
options = Options()
options.add_argument('--headless')  # Run in headless mode (without GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')  # Set window size to handle elements properly

# Setup the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL of the Flipkart page you want to scrape
url = 'https://www.flipkart.com/search?q=laptop'

# Navigate to the page
driver.get(url)

# Initialize an empty list to store product details
products = []

# Function to extract product details from the current page
def extract_product_details(soup):
    cards = soup.find_all('div', class_='tUxRFH')  # Adjust this if the structure changes

    for card in cards:
        try:
            model = card.find('div', {'class': "KzDlHZ"}).text
            star = card.find('div', {'class': "XQDdHH"}).text
            num_ratings = card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ") \
                [0:card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ").find(';')].strip()
            reviews = card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ") \
                [card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ").find(';') + 1:].strip()
            processor = card.find_all('li')[0].text.strip()
            ram = card.find_all('li')[1].text.strip()
            os = card.find_all('li')[2].text.strip()
            storage = card.find_all('li')[3].text.strip()
            size = card.find_all('li')[4].text.strip()
            price = card.find('div', {'class': 'Nx9bqj _4b5DiR'}).text.strip()

            # Append product details to the products list
            products.append({
                'Model': model,
                'Star Rating': star,
                'Number of Ratings': num_ratings,
                'Number of Reviews': reviews,
                'Processor': processor,
                'RAM': ram,
                'OS': os,
                'Storage': storage,
                'Size': size,
                'Price': price
            })
        except Exception as e:
            print(f"Error processing card: {e}")

# Scrape multiple pages
while True:
    # Retrieve page source
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract product details from the current page
    extract_product_details(soup)

    try:
        next_button = driver.find_element('xpath', "//*[@id='container']/div/div[3]/div[1]/div[2]/div[12]/div/div/nav/a[11]")
        next_button.click()
        time.sleep(3)  # Wait for the next page to load
    except:
        print("No more pages or unable to find the Next button.")
        break

# Close the driver
driver.quit()

# Convert the list of products into a DataFrame
df = pd.DataFrame(products)

# Save the DataFrame to a CSV file
df.to_csv('flipkart_laptopsc.csv', index=False)

print("Data has been saved to 'flipkart_gaming_laptops.csv'.")


KeyboardInterrupt: 

In [104]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd  # Import pandas for saving the data

# Setup Chrome options
options = Options()
options.add_argument('--headless')  # Run in headless mode (without GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')  # Set window size to handle elements properly

# Setup the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL of the Flipkart page you want to scrape
url = 'https://www.flipkart.com/search?q=laptop'

# Navigate to the page
driver.get(url)

# Initialize an empty list to store product details
products = []

# Function to extract product details from the current page
def extract_product_details(soup):
    cards = soup.find_all('div', class_='tUxRFH')  # Adjust this if the structure changes

    for card in cards:
        try:
            model = card.find('div', {'class': "KzDlHZ"}).text if card.find('div', {'class': "KzDlHZ"}) else 'N/A'
            star = card.find('div', {'class': "XQDdHH"}).text if card.find('div', {'class': "XQDdHH"}) else 'No rating'
            num_ratings = card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ") \
                [0:card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ").find(';')].strip() \
                if card.find('span', {'class': "hG7V+4"}) else 'No ratings'
            reviews = card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ") \
                [card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ").find(';') + 1:].strip() \
                if card.find('span', {'class': "hG7V+4"}) else 'No reviews'
            processor = card.find_all('li')[0].text.strip() if len(card.find_all('li')) > 0 else 'N/A'
            ram = card.find_all('li')[1].text.strip() if len(card.find_all('li')) > 1 else 'N/A'
            os = card.find_all('li')[2].text.strip() if len(card.find_all('li')) > 2 else 'N/A'
            storage = card.find_all('li')[3].text.strip() if len(card.find_all('li')) > 3 else 'N/A'
            size = card.find_all('li')[4].text.strip() if len(card.find_all('li')) > 4 else 'N/A'
            price = card.find('div', {'class': 'Nx9bqj _4b5DiR'}).text.strip() if card.find('div', {'class': 'Nx9bqj _4b5DiR'}) else 'Price not available'

            # Append product details to the products list
            products.append({
                'Model': model,
                'Star Rating': star,
                'Number of Ratings': num_ratings,
                'Number of Reviews': reviews,
                'Processor': processor,
                'RAM': ram,
                'OS': os,
                'Storage': storage,
                'Size': size,
                'Price': price
            })
        except Exception as e:
            print(f"Error processing card: {e}")

# Scrape multiple pages
while True:
    # Retrieve page source
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract product details from the current page
    extract_product_details(soup)

    try:
        next_button = driver.find_element('xpath', "//*[@id='container']/div/div[3]/div[1]/div[2]/div[12]/div/div/nav/a[11]")
        next_button.click()
        time.sleep(3)  # Wait for the next page to load
    except:
        print("No more pages or unable to find the Next button.")
        break

# Close the driver
driver.quit()

# Convert the list of products into a DataFrame
df = pd.DataFrame(products)

# Save the DataFrame to a CSV file
df.to_csv('flipkart_gaming_laptops.csv', index=False)

print("Data has been saved to 'flipkart_gaming_laptops.csv'.")


No more pages or unable to find the Next button.
Data has been saved to 'flipkart_gaming_laptops.csv'.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd  # Import pandas for saving the data

# Setup Chrome options
options = Options()
options.add_argument('--headless')  # Run in headless mode (without GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')  # Set window size to handle elements properly

# Setup the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL of the Flipkart page you want to scrape
url = 'https://www.flipkart.com/search?q=books&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=10'

# Navigate to the page
driver.get(url)

# Initialize an empty list to store product details
products = []
max_rows = 4000  # Limit for the number of rows to scrape

# Function to extract product details from the current page
def extract_product_details(soup):
    cards = soup.find_all('div', class_='tUxRFH')  # Adjust this if the structure changes

    for card in cards:
        try:
            model = card.find('div', {'class': "KzDlHZ"}).text if card.find('div', {'class': "KzDlHZ"}) else 'N/A'
            star = card.find('div', {'class': "XQDdHH"}).text if card.find('div', {'class': "XQDdHH"}) else 'No rating'
            num_ratings = card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ") \
                [0:card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ").find(';')].strip() \
                if card.find('span', {'class': "hG7V+4"}) else 'No ratings'
            reviews = card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ") \
                [card.find('span', {'class': "hG7V+4"}).text.replace('\xa0&\xa0', " ; ").find(';') + 1:].strip() \
                if card.find('span', {'class': "hG7V+4"}) else 'No reviews'
            processor = card.find_all('li')[0].text.strip() if len(card.find_all('li')) > 0 else 'N/A'
            ram = card.find_all('li')[1].text.strip() if len(card.find_all('li')) > 1 else 'N/A'
            os = card.find_all('li')[2].text.strip() if len(card.find_all('li')) > 2 else 'N/A'
            storage = card.find_all('li')[3].text.strip() if len(card.find_all('li')) > 3 else 'N/A'
            size = card.find_all('li')[4].text.strip() if len(card.find_all('li')) > 4 else 'N/A'
            price = card.find('div', {'class': 'Nx9bqj _4b5DiR'}).text.strip() if card.find('div', {'class': 'Nx9bqj _4b5DiR'}) else 'Price not available'

            # Append product details to the products list
            products.append({
                'Model': model,
                'Star Rating': star,
                'Number of Ratings': num_ratings,
                'Number of Reviews': reviews,
                'Processor': processor,
                'RAM': ram,
                'OS': os,
                'Storage': storage,
                'Size': size,
                'Price': price
            })
        except Exception as e:
            print(f"Error processing card: {e}")

# Scrape multiple pages
while True:
    # Retrieve page source
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract product details from the current page
    extract_product_details(soup)

    # Check if we have reached the limit
    if len(products) >= max_rows:
        print("Reached the maximum row limit of 2000.")
        break

    try:
        # Locate the next page button using its class name
        next_buttons = driver.find_elements('a', '_9QVEpD')  # Find all elements with class '_9QVEpD'
        if next_buttons:
            next_buttons[-1].click()  # Click on the last button (next page)
            time.sleep(3)  # Wait for the next page to load
        else:
            print("No more pages or unable to find the Next button.")
            break
    except Exception as e:
        print(f"Error while navigating to next page: {e}")
        break

# Close the driver
driver.quit()

# Convert the list of products into a DataFrame
df = pd.DataFrame(products)

# Save the DataFrame to a CSV file
df.to_csv('books.csv', index=False)

print("Data has been saved to 'books.csv'.")


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Setup Chrome options
options = Options()
options.add_argument('--headless')  # Run in headless mode (without GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')  # Set window size to handle elements properly

# Setup the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL of the Flipkart page you want to scrape
url = 'https://www.flipkart.com/search?q=laptops&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off'

# Navigate to the page
driver.get(url)

# Function to extract product details from the current page
def extract_product_details(soup):
    cards = soup.find_all('div', {'data-component-type': 's-search-result'})  # Find all product cards

    for card in cards:
        # Extract product name
        product_name = card.find('span', class_='a-size-medium a-color-base a-text-normal')
        
        # Extract product price
        product_price_whole = card.find('span', class_='a-price-whole')  # Price (whole part)
        product_price_fraction = card.find('span', class_='a-price-fraction')  # Price (fraction part)
        product_price = f"{product_price_whole.text.strip()}.{product_price_fraction.text.strip()}" if product_price_whole and product_price_fraction else None
        
        # Extract product description (if available)
        product_desc = card.find('span', class_='a-size-base-plus')  # Adjust this as needed for specific description fields
        
        # Extract product rating
        product_rating = card.find('span', class_='a-icon-alt')  # Ratings appear here
        
        # Extract number of reviews
        product_no_rev = card.find('span', class_='a-size-base')  # Number of reviews container (may need refining)

        if product_name:
            print(f"Product Name: {product_name.text.strip()}")
            print(f"Product Price: {product_price if product_price else 'N/A'}")
            print(f"Product Description: {product_desc.text.strip() if product_desc else 'N/A'}")
            print(f"Product Rating: {product_rating.text.strip() if product_rating else 'No rating'}")
            print(f"No. of Product Reviews: {product_no_rev.text.strip() if product_no_rev else 'No reviews'}")
            print('-' * 80)

# Scrape multiple pages
while True:
    # Retrieve page source
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract product details from the current page
    extract_product_details(soup)

    try:
        next_button = driver.find_element('xpath', "//*[@id='container']/div/div[3]/div[1]/div[2]/div[12]/div/div/nav/a[11]")
        next_button.click()
        time.sleep(3)  # Wait for the next page to load
    except:
        print("No more pages or unable to find the Next button.")
        break

# Close the driver
driver.quit()

# Convert the list of products into a DataFrame
df = pd.DataFrame(products)

# Save the DataFrame to an Excel file
df.to_excel('flipkart_laptops.xlsx', index=False)

print("Data has been saved to 'flipkart_laptops.xlsx'.")

KeyboardInterrupt: 